# 星巴克毕业项目

# 数据集

一共有三个数据文件：

* portfolio.json – 包括推送的 id 和每个推送的元数据（持续时间、种类等等）
* profile.json – 每个顾客的人口统计数据
* transcript.json – 交易、收到的推送、查看的推送和完成的推送的记录

以下是文件中每个变量的类型和解释 ：

**portfolio.json**
* id (string) – 推送的id
* offer_type (string) – 推送的种类，例如 BOGO、打折（discount）、信息（informational）
* difficulty (int) – 满足推送的要求所需的最少花费
* reward (int) – 满足推送的要求后给与的优惠
* duration (int) – 推送持续的时间，单位是天
* channels (字符串列表)

**profile.json**
* age (int) – 顾客的年龄 
* became_member_on (int) – 该顾客第一次注册app的时间
* gender (str) – 顾客的性别（注意除了表示男性的 M 和表示女性的 F 之外，还有表示其他的 O）
* id (str) – 顾客id
* income (float) – 顾客的收入

**transcript.json**
* event (str) – 记录的描述（比如交易记录、推送已收到、推送已阅）
* person (str) – 顾客id
* time (int) – 单位是小时，测试开始时计时。该数据从时间点 t=0 开始
* value - (dict of strings) – 推送的id 或者交易的数额

# <a class="anchor" id="Table-Start">Table of Contents</a>

I. [Data Exploration](#Data-Exploration)<br>
II.[Data Cleaning](#Data-Cleaning)<br>
III.[Data Preprocessing](#Data-Preprocessing)<br>
　　3.1. [Wrangle Data](#Wrangle-Data)<br>
　　　　Method.1 [Using a moduled class](#Method-1-wrangle)<br>
　　　　Method.2 [Based on following functions](#Method-2-wrangle) (better for debugging)<br>
　　3.2. [Save the wrangled data](#Save-Data)<br>
IV.[Explore the wrangled Data](#Wrangled-Data)(preliminary)<br>
[References](#References)

### Imports & Load in data

In [1]:
import pandas as pd
import numpy as np
import math
import json

from time import time
from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# <a class="anchor" id="Data-Exploration">I. [Data Exploration](#Table-Start)</a>

## 1.1 Basic Information
- .head()
    - Index, Columns(Features)
- .info()
    - NaN values, data size
- .describe()
    - Statistik infomation
- .unique()
- .value_counts()

In [2]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
portfolio.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [4]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


**FOUND**:
- the type of 'became_member_on' is int64

In [6]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


**FOUND**:
- the max age is 118

In [7]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [8]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


# <a class="anchor" id="Data-Cleaning">II. [Data Cleaning](#Table-Start)</a>
### 2.1. Demographic data: `profile`

#### `2.1.1.` Encode the column 'id'
(related to DataFrame `transcript`, `id` Encoding should done before .dropna())
1. Map 'id' to int number

In [9]:
# set the dict{"original Encoding": int}
id_2_int = defaultdict(int)
encoding_ids = profile['id'].unique()
num = 0
for encoding_id in encoding_ids:
    id_2_int[encoding_id] = num
    num += 1
    
def person_id_mapping(df_2b_mapped, col, mapping_dict=id_2_int):
    '''Map 'id' from Encoding to int 
    
    INPUT:
        df_2b_mapped - (DataFrame) the target DataFrame in which 'id' to be mapped
        col - (str) the target col ('id') to be mapped
        mapping_dict - (dict) mapping dict {"original Encoding": int}
    
    OUTPUT:
        df_2_map - (DataFrame) already mapped DataFrame
    '''
    df_2b_mapped[col] = df_2b_mapped[col].apply(lambda x: id_2_int[x])
    
    return df_2b_mapped

In [10]:
# map 'id' in profile_cleaned from Encoding to int
profile_2_clean = person_id_mapping(profile, 'id')
profile_2_clean

,gender,age,id,became_member_on,income
0,None,118,0,20170212,NaN
1,F,55,1,20170715,112000.0
2,None,118,2,20180712,NaN
3,F,75,3,20170509,100000.0
4,None,118,4,20170804,NaN
...,...,...,...,...,...
16995,F,45,16995,20180604,54000.0
16996,M,61,16996,20180713,72000.0
16997,M,49,16997,20170126,73000.0
16998,F,83,16998,20160307,50000.0



#### `2.1.2.` NaNs
1. The NaNs in columns 'gender', 'income' and illegal age(age is 118) appear simultaneously.
    - It's fine to delete all the 2175 NaNs.

In [11]:
# get the NaN and illegal values in 'income', 'gender', 'age'
idx_nan_in_income = np.where(profile_2_clean.income.isnull())[0]
idx_none_in_gender = np.where(profile_2_clean.gender.isnull())[0]
idx_illegal_age = np.where(profile_2_clean.age==118)

In [12]:
# the illegal values appear in the same index 
print((idx_nan_in_income  == idx_none_in_gender).all())
print((idx_nan_in_income  == idx_illegal_age).all())

True
True


In [30]:
# drop all 2175 NaNs
profile_cleaned = profile_2_clean.dropna(axis=0)
profile_cleaned

(14825, 5)


,gender,age,id,became_member_on,income
1,F,55,1,20170715,112000.0
3,F,75,3,20170509,100000.0
5,M,68,5,20180426,70000.0
8,M,65,8,20180209,53000.0
12,M,58,12,20171111,51000.0
...,...,...,...,...,...
16995,F,45,16995,20180604,54000.0
16996,M,61,16996,20180713,72000.0
16997,M,49,16997,20170126,73000.0
16998,F,83,16998,20160307,50000.0


### 2.2. Promotion offer data: `portfolio`

#### `2.2.1.` Encode the column 'id'
1. Map 'id' to int number

In [14]:
#set the dict{"original Encoding": int}
offer_id_2_int = defaultdict()
offer_ids = portfolio['id']
label = 0
for offer_id in offer_ids:
    offer_id_2_int[offer_id]=label
    label += 1

# later, we use .fillna(-1) to replace the nan in 'offer_id' in transcript_cleaned
offer_id_2_int[-1]=-1

offer_id_2_int

defaultdict(None,
            {'ae264e3637204a6fb9bb56bc8210ddfd': 0,
             '4d5c57ea9a6940dd891ad53e9dbe8da0': 1,
             '3f207df678b143eea3cee63160fa8bed': 2,
             '9b98b8c7a33c4b65b9aebfe6a799e6d9': 3,
             '0b1e1539f2cc45b7b9fa7c272da2e1d7': 4,
             '2298d6c36e964ae4a3e7e9706d1fb8c2': 5,
             'fafdcd668e3743c1bb461111dcafc2a4': 6,
             '5a8bc65990b245e5a138643cd4eb9837': 7,
             'f19421c1d4aa40978ebb69ca19b0e20d': 8,
             '2906b810c7d4411798c6938adc9daaa5': 9,
             -1: -1})

In [15]:
portfolio_2_clean = portfolio.copy()
portfolio_2_clean['id'] = portfolio['id'].apply(lambda x: offer_id_2_int[x])
portfolio_2_clean

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,0
1,10,"[web, email, mobile, social]",10,5,bogo,1
2,0,"[web, email, mobile]",0,4,informational,2
3,5,"[web, email, mobile]",5,7,bogo,3
4,5,"[web, email]",20,10,discount,4
5,3,"[web, email, mobile, social]",7,7,discount,5
6,2,"[web, email, mobile, social]",10,10,discount,6
7,0,"[email, mobile, social]",0,3,informational,7
8,5,"[web, email, mobile, social]",5,5,bogo,8
9,2,"[web, email, mobile]",10,7,discount,9


#### `2.2.2.` Column: 'channels'
1. Extract the list values into dependent columns

In [16]:
# References[1]
# Extract the list values into dependent columns
channels_2_df = portfolio_2_clean['channels'].str.join(sep='*').str.get_dummies(sep='*')

In [17]:
# concat the extracted data from 'channels' with the original data, and delete 'channels'
portfolio_2_clean = pd.concat([portfolio_2_clean, channels_2_df], axis=1)
portfolio_cleaned= portfolio_2_clean.drop(labels='channels', axis=1)
portfolio_cleaned

(10, 9)


### 2.3. Transactional records: `transcript`
#### `2.3.1.` Encoding the column 'person'
1. map 'person' from Encoding to int
2. update synchronous to dataset `profile_cleaned`(some person_id were dropped)

In [18]:
# map 'person' in transcript_cleaned from Encoding to int
# there are the same number of unique person in both transcript and profile: 17,000
transcript_2_clean = person_id_mapping(transcript, 'person')

In [19]:
# update synchronous to dataset profile_cleaned(some person_id were dropped)
valid_person = profile_cleaned.id.unique()
is_valid_person = transcript_2_clean.person.isin(profile_cleaned.id.unique())
transcript_2_clean = transcript_2_clean[is_valid_person]

print("number of unique person at begin is {}" .format(transcript.person.nunique()))
print("number of unique person synchronous updated is {}" .format(transcript_2_clean.person.nunique()))
transcript_2_clean #before size (306534,4)

number of unique person at begin is 17000
number of unique person synchronous updated is 14825


,person,event,value,time
0,3,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
2,5,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
5,8,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
7,12,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},0
8,13,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
...,...,...,...,...
306527,16949,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
306529,16959,transaction,{'amount': 1.5899999999999999},714
306530,16964,transaction,{'amount': 9.53},714
306531,16979,transaction,{'amount': 3.61},714


#### `2.3.2.` Extract the column: 'value'
1. Extract the dict values into dependent columns according to the keys in dict

In [20]:
# Extract the dict values into dependent columns
value_2_df = transcript_2_clean.value.apply(pd.Series)

In [21]:
# Combine the data in 'offer id' into 'offer_id' (they belong to same data: but in wrong name)
is_offer_id_2b_updated = value_2_df['offer id'].notnull()
value_2_df['offer_id'] = np.where(is_offer_id_2b_updated, value_2_df['offer id'], value_2_df['offer_id'])

In [22]:
# in original data: old 'offer_id' appears together with 'reward'
# check: number of new 'offer_id' == (number of 'reward') +  (number of 'offer id')
print("The number of data in 'offer id' is {}"  .format(value_2_df['offer id'].count()))
print("The number of data in 'offer_id' is {}"  .format(value_2_df['reward'].count())) # before updated, bounded with 'reward'
print("The number of data in updated 'offer_id' is {} == {}+{}"  .format(value_2_df['offer_id'].count(), value_2_df['offer id'].count(), value_2_df['reward'].count()))

The number of data in 'offer id' is 116361
The number of data in 'offer_id' is 32444
The number of data in updated 'offer_id' is 148805 == 116361+32444


In [23]:
# drop the column 'offer id'
value_2_df.drop(labels='offer id', axis=1, inplace=True)
value_2_df

,amount,offer_id,reward
0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
2,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
5,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
7,NaN,3f207df678b143eea3cee63160fa8bed,NaN
8,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
...,...,...,...
306527,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0
306529,1.59,NaN,NaN
306530,9.53,NaN,NaN
306531,3.61,NaN,NaN


In [24]:
# concat value_2_df to the original dataset transcript
# drop the two useless columns 'value', 'reward'
transcript_2_clean = pd.concat([transcript_2_clean, value_2_df], axis=1).drop(labels=['value','reward'], axis=1)

#### `2.3.3.` NaNs 
1. use .fillna()
    - in column 'amount': replace NaNs with `0`(float)
    - in column 'offer_id': replace NaNs with `-1`(int), then use map dict replace the -1(int) to -1(int)

In [25]:
# set the NaNs to special values
transcript_2_clean.fillna(value={'amount': 0, 'offer_id': -1}, axis =0, inplace=True)
transcript_2_clean['offer_id'] = transcript_2_clean['offer_id'].apply(lambda x: offer_id_2_int[x])

#### `2.3.4.` change unit of 'time' from hours to days

In [26]:
transcript_2_clean['time']=transcript_2_clean['time']/24.0

In [27]:
transcript_cleaned = transcript_2_clean.copy()
transcript_cleaned

,person,event,time,amount,offer_id
0,3,offer received,0.00,0.00,3
2,5,offer received,0.00,0.00,9
5,8,offer received,0.00,0.00,8
7,12,offer received,0.00,0.00,2
8,13,offer received,0.00,0.00,4
...,...,...,...,...,...
306527,16949,offer completed,29.75,0.00,6
306529,16959,transaction,29.75,1.59,-1
306530,16964,transaction,29.75,9.53,-1
306531,16979,transaction,29.75,3.61,-1


### 2.4. Prepare the data to wrangle and Save to .csv files
`2.4.1.` Merge Dataset
- Merge `transcript_cleaned` with details of 'offer_id' in `portfolio_cleaned`
- Change the type of 'offer_id' to `str`(automatically changed to `object`)

In [28]:
# replace column 'id' with the unite column name 'offer_id'
portfolio_cleaned.rename({'id': 'offer_id'},axis=1, inplace=True)

In [29]:
# Merge transcript_cleaned with details of 'offer_id' in portfolio_cleaned 
transcript_offer = transcript_cleaned.merge(portfolio_cleaned[['duration','offer_id','offer_type']], how='left', on='offer_id')

transcript_offer = transcript_offer.astype({'offer_id': 'str'})
#transcript_offer = transcript_offer.astype({'person': 'int'}).astype({'person': 'object'})

transcript_offer

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272762 entries, 0 to 272761
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   person      272762 non-null  int64  
 1   event       272762 non-null  object 
 2   time        272762 non-null  float64
 3   amount      272762 non-null  float64
 4   offer_id    272762 non-null  object 
 5   duration    148805 non-null  float64
 6   offer_type  148805 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 16.6+ MB


`2.4.2.` Save Dataset 

```python 
profile_cleaned.to_csv('./cleaned_profile.csv') #如果与index不相关的表格 可以设置index=False
portfolio_cleaned.to_csv('./cleaned_portfolio.csv')
transcript_offer.to_csv('./transcript_offer.csv')
```

# <a class="anchor" id="Data-Preprocessing">III. [Data Preprocessing](#Table-Start)</a>
Follow program flow chart:
<img src="./images/flow_chart.jpg"/>
We get two ideal dataset:
I. target_dataset
<img src="./images/target_dataset.png"/>
1. `person` (int)
    - Id for person: already encoded to int number
2. `offer_id` (object of str)
    - Id for offer: already encoded to int number
    - **Values**: from '0' to '9'
3. `time_received` (float)
    - time of reveiving the offer
    - **Values**: `NaN` represent not received
4. `time_viewed` (float) 
    - time of viewing the offer
    - **Values**: `NaN` represent not viewed
5. `time_transaction` (object of str)
    - time of the transactions within an transaction unit(within one unique offer_id of one person, there may be more transaction units)
    - **Values**: there maybe more than one transactions within a transaction unit, so use `str`(obeject) to represent
        - `''` means there is no transaction
        - `',3.8,5.9'` means there is two transactions, one is at time 3.8, another in at time 5.9
6. `time_completed` (float)
    - time of complete the offer
    - **Values**: `NaN` represent not completed
7. `amount_with_offer` (float)
    - amount of transaction(s) within this transaction unit
    - **Values**: '0.0' represent no transaction
8. `label_effective_offer` (int)
    - the label to mark the completion level of offer
    - **Values**:
        - `1`: 
            - for informational offer there is at least one transaction within duration; 
            - for other offer there should be 'offer completed'
        - `0`: 
            - for informational offer there is no transaction but 'offer received'; 
            - for other offers there is no 'offer completed', but within duration there maybe some amount, although the amount of transactions not fulfil requirements
        - `-1`: the init label, when there is no 'offer received', the label keeps '-1'
        - `-2`: Special, some interesting discovering after the data was wrangled
            - represent some people: they only have transactions within all the experimantal time , no offer was sent to them

II. transcript_offer(updated)<br>

```python
### Code
# Just show the updated part:
normal_offer_id = target_dataset.offer_id.unique().tolist()   # ['0','1','2','3','4','5','6','7','8','9']
# show transaction, the offer_id is not normal(some has been updated by wrangling)
transcript_offer[~transcript_offer.offer_id.isin(normal_offer_id)].tail(20)
```
<img src="./images/transcript_offer.png"/>
most are the same as the original dataset, just some values in column 'offer_id' are updated <br>
in figure, the value '6,5' represent:<br>
　　this transaction is valid for both of the two offer_ids: one is '6', another is '5'.

## <a class="anchor" id="Wrangle-Data">3.1. [Wrangle Data](#Table-Start)</a>
**Goal**: get two dataset
1. Ideal target_dataset 
    - initial a target_dataset_list
    - append list with constructed dicts
    - change target_dataset_list to DataFrame target_dataset
2. Update transcript_offer
    - Update the offer_id infos of transactions with the related offer_id in original dataset transcript_offer
3. There are **already two wrangled datasets**
    - `wrangled_transcript_offer.csv`
    - `wrangled_target_dataset.csv`

In [36]:
# prepare two dataset to wrangle

# 1. target_dataset_list 
target_dataset_list = []
# pd.DataFrame(columns=['person', 'offer_id', 'time_received', 'time_viewed', 'time_transaction','time_completed','amount_with_offer','label_effective_offer'])

# 2. transcript_offer
transcript_offer # already existed

,person,event,time,amount,offer_id,duration,offer_type
0,3,offer received,0.00,0.00,3,7.0,bogo
1,5,offer received,0.00,0.00,9,7.0,discount
2,8,offer received,0.00,0.00,8,5.0,bogo
3,12,offer received,0.00,0.00,2,4.0,informational
4,13,offer received,0.00,0.00,4,10.0,discount
...,...,...,...,...,...,...,...
272757,16949,offer completed,29.75,0.00,6,10.0,discount
272758,16959,transaction,29.75,1.59,-1,NaN,NaN
272759,16964,transaction,29.75,9.53,-1,NaN,NaN
272760,16979,transaction,29.75,3.61,-1,NaN,NaN


In [ ]:
'''
### or Load the already saved dataset

transcript_offer = pd.read_csv('./transcript_offer.csv', dtype={'offer_id': str})

# recover to original dataset: index is the same
transcript_offer.index = transcript_offer.iloc[:, 0].values
del transcript_offer['Unnamed: 0']
'''

# !!!Attention: The Wrangling takes more than 2 hours...
- The source code for Wrangling are .py files in project
    - class: `data_preprocessing_class.py`
    - functions: `data_preprocessing_funcs.py`

### <a class="anchor" id="Method-1-wrangle">[Method.1 Using a moduled class](#Table-Start)</a>
- in `data_preprocessing_class.py` <br>

    ```python
    from data_preprocessing_class import DataPreprocessing
    ```

In [39]:
# import the class from python files
from data_preprocessing_class import PreprocessingData

In [40]:
data_process = PreprocessingData(target_dataset_list, transcript_offer)

In [41]:
# get the unique_person list
person_ids = transcript_offer.person.unique()
for person_id in person_ids:
    # transcript for unique_person
    unique_person = transcript_offer[transcript_offer['person']==person_id]
    # all transactions of unique_person
    transactions = unique_person[unique_person.event=='transaction']
    
    # get groupby object according to 'offer_id' of a unique_person
    groupby_offer_id = unique_person.groupby(['offer_id'])
    offer_id_list = unique_person.offer_id.unique()
    
    # update target_data_list and transcript_offer
    data_process.get_dateset_from_unique_offer_id(groupby_offer_id, offer_id_list, transactions)

D:\JustPractice\1_MS_Geek_salute\PythonAnaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


data for unique person wrangled, time is 0.6656148433685303
data for unique person wrangled, time is 0.42074084281921387
data for unique person wrangled, time is 1.0064234733581543
data for unique person wrangled, time is 0.34780144691467285
data for unique person wrangled, time is 0.6596226692199707
data for unique person wrangled, time is 0.8555114269256592
data for unique person wrangled, time is 0.7485699653625488
data for unique person wrangled, time is 0.6976020336151123
data for unique person wrangled, time is 0.5836482048034668
data for unique person wrangled, time is 0.8375194072723389
data for unique person wrangled, time is 0.9964301586151123
data for unique person wrangled, time is 0.8195102214813232
data for unique person wrangled, time is 0.8365194797515869
data for unique person wrangled, time is 0.6136460304260254
data for unique person wrangled, time is 0.896486759185791
data for unique person wrangled, time is 0.8474936485290527
data for unique person wrangled, time i

data for unique person wrangled, time is 0.8884932994842529
data for unique person wrangled, time is 0.5246975421905518
data for unique person wrangled, time is 0.47070980072021484
data for unique person wrangled, time is 1.366218090057373
data for unique person wrangled, time is 0.9504566192626953
data for unique person wrangled, time is 0.597658634185791
data for unique person wrangled, time is 1.1173617839813232
data for unique person wrangled, time is 1.2063093185424805
data for unique person wrangled, time is 1.024411678314209
data for unique person wrangled, time is 0.48270344734191895
data for unique person wrangled, time is 1.1473417282104492
data for unique person wrangled, time is 0.9314661026000977
data for unique person wrangled, time is 0.5496673583984375
data for unique person wrangled, time is 0.8924903869628906
data for unique person wrangled, time is 0.7735540866851807
data for unique person wrangled, time is 0.5856459140777588
data for unique person wrangled, time is 

data for unique person wrangled, time is 0.8285248279571533
data for unique person wrangled, time is 0.6396534442901611
data for unique person wrangled, time is 0.7655627727508545
data for unique person wrangled, time is 0.41974353790283203
data for unique person wrangled, time is 0.32781004905700684
data for unique person wrangled, time is 0.9014627933502197
data for unique person wrangled, time is 0.561678409576416
data for unique person wrangled, time is 0.31380152702331543
data for unique person wrangled, time is 0.8645038604736328
data for unique person wrangled, time is 0.47872209548950195
data for unique person wrangled, time is 0.705594539642334
data for unique person wrangled, time is 0.24486255645751953
data for unique person wrangled, time is 0.6566243171691895
data for unique person wrangled, time is 0.5656583309173584
data for unique person wrangled, time is 0.9654481410980225
data for unique person wrangled, time is 0.7395751476287842
data for unique person wrangled, time

data for unique person wrangled, time is 0.7225878238677979
data for unique person wrangled, time is 0.8515124320983887
data for unique person wrangled, time is 0.35477638244628906
data for unique person wrangled, time is 1.1203577518463135
data for unique person wrangled, time is 0.33580923080444336
data for unique person wrangled, time is 0.5886452198028564
data for unique person wrangled, time is 0.5286967754364014
data for unique person wrangled, time is 0.5786480903625488
data for unique person wrangled, time is 0.6196465492248535
data for unique person wrangled, time is 0.7705767154693604
data for unique person wrangled, time is 0.7695586681365967
data for unique person wrangled, time is 0.7635622024536133
data for unique person wrangled, time is 0.6546266078948975
data for unique person wrangled, time is 0.7615628242492676
data for unique person wrangled, time is 0.7415783405303955
data for unique person wrangled, time is 0.45873498916625977
data for unique person wrangled, time

...

data for unique person wrangled, time is 0.5906434059143066
data for unique person wrangled, time is 0.44672489166259766
data for unique person wrangled, time is 0.15790891647338867
data for unique person wrangled, time is 0.3277928829193115
data for unique person wrangled, time is 0.6756107807159424
data for unique person wrangled, time is 0.5456700325012207
data for unique person wrangled, time is 0.7775554656982422
data for unique person wrangled, time is 0.5097081661224365
data for unique person wrangled, time is 0.5556623935699463
data for unique person wrangled, time is 0.4617350101470947
data for unique person wrangled, time is 0.6096532344818115
data for unique person wrangled, time is 0.5656578540802002
data for unique person wrangled, time is 0.6765942573547363
data for unique person wrangled, time is 0.7025971412658691
data for unique person wrangled, time is 0.35379695892333984
data for unique person wrangled, time is 0.8635048866271973
data for unique person wrangled, time

data for unique person wrangled, time is 0.6116509437561035
data for unique person wrangled, time is 0.6166286468505859
data for unique person wrangled, time is 0.4157423973083496
data for unique person wrangled, time is 0.8185315132141113
data for unique person wrangled, time is 0.6946022510528564
data for unique person wrangled, time is 0.48972225189208984
data for unique person wrangled, time is 0.5906405448913574
data for unique person wrangled, time is 0.7216048240661621
data for unique person wrangled, time is 0.5796668529510498
data for unique person wrangled, time is 0.4767265319824219
data for unique person wrangled, time is 0.43672895431518555
data for unique person wrangled, time is 0.45174384117126465
data for unique person wrangled, time is 0.5416710376739502
data for unique person wrangled, time is 0.43775153160095215
data for unique person wrangled, time is 0.3048069477081299
data for unique person wrangled, time is 0.4737095832824707
data for unique person wrangled, tim

data for unique person wrangled, time is 0.47872495651245117
data for unique person wrangled, time is 0.24683928489685059
data for unique person wrangled, time is 0.443727970123291
data for unique person wrangled, time is 0.4207580089569092
data for unique person wrangled, time is 0.30784106254577637
data for unique person wrangled, time is 0.4977147579193115
data for unique person wrangled, time is 0.6376326084136963
data for unique person wrangled, time is 0.3428027629852295
data for unique person wrangled, time is 0.42975282669067383
data for unique person wrangled, time is 0.535693883895874
data for unique person wrangled, time is 0.45471835136413574
data for unique person wrangled, time is 0.4557209014892578
data for unique person wrangled, time is 0.4437272548675537
data for unique person wrangled, time is 0.1619272232055664
data for unique person wrangled, time is 0.40674829483032227
data for unique person wrangled, time is 0.4247777462005615
data for unique person wrangled, tim

data for unique person wrangled, time is 0.25585269927978516
data for unique person wrangled, time is 0.32980799674987793
data for unique person wrangled, time is 0.4867238998413086
data for unique person wrangled, time is 0.46573328971862793
data for unique person wrangled, time is 0.4417436122894287
data for unique person wrangled, time is 0.47670912742614746
data for unique person wrangled, time is 0.3157987594604492
data for unique person wrangled, time is 0.6456294059753418
data for unique person wrangled, time is 0.5706734657287598
data for unique person wrangled, time is 0.5596787929534912
data for unique person wrangled, time is 0.31381988525390625
data for unique person wrangled, time is 0.32679200172424316
data for unique person wrangled, time is 0.5226812362670898
data for unique person wrangled, time is 0.4667325019836426
data for unique person wrangled, time is 0.317798376083374
data for unique person wrangled, time is 0.3467998504638672
data for unique person wrangled, ti

data for unique person wrangled, time is 0.506711483001709
data for unique person wrangled, time is 0.47272753715515137
data for unique person wrangled, time is 0.31182050704956055
data for unique person wrangled, time is 0.468733549118042
data for unique person wrangled, time is 0.29581165313720703
data for unique person wrangled, time is 0.4087667465209961
data for unique person wrangled, time is 0.17390108108520508
data for unique person wrangled, time is 0.32780933380126953
data for unique person wrangled, time is 0.49071693420410156
data for unique person wrangled, time is 0.4507412910461426
data for unique person wrangled, time is 0.3348085880279541
data for unique person wrangled, time is 0.38779568672180176
data for unique person wrangled, time is 0.08495116233825684
data for unique person wrangled, time is 0.48372530937194824
data for unique person wrangled, time is 0.39475512504577637
data for unique person wrangled, time is 0.30182671546936035
data for unique person wrangled

In [42]:
# change target_dataset_list(list of dicts) to DataFrame
target_dataset = pd.DataFrame(data_process.target_dataset_list, 
                              columns=['person', 'offer_id', 'time_received', 'time_viewed', 'time_transaction','time_completed','amount_with_offer','label_effective_offer'])
# update transcript_offer
transcript_offer = data_process.transcript_offer

### <a class="anchor" id="Method-2-wrangle">[Method.2 Based on following functions](#Table-Start) (better for debugging)</a>
- in `data_preprocessing_funcs.py`
    - get_dateset_from_unique_offer_id(groupby_offer_id, offer_id_list, transactions) -> None
    - fill_offer_id_4_transaction(original_id, updated_id) -> lst
    - get_dateset_from_informational_offer(df_units, units_count, transactions) -> None
    - get_dateset_from_other_offer(df_units, units_count, transactions) -> None
    - cut_unique_offer_id_2_units(unique_offer_id) -> df_units, units_count

In [ ]:
def get_dateset_from_unique_offer_id(groupby_offer_id, offer_id_list, transactions):
    '''
    DESCRIPTION:
    Based on unique_offer_id of unique_person update the following dataset:

    Update transactions with related offer_id in the original dataset transcript_offer.

    Update target_dataset_list - (list), the structure of target_dataset_list element is a dict with keys
        'person', 'offer_id', 'time_received', 'time_viewed', 'time_transaction','time_completed','amount_with_offer','label_effective_offer'

    INPUT:
        groupby_offer_id - (pandas groupby object) groupby offer_id of get_dateset_from_unique_person
        offer_id_list - (list) offer_id list with unique values of unique person
        transactions - (DataFrame) all transactions of a unique person

    OUTPUT: None
    '''
    start = time()

    valid_offer_id = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for offer_id in offer_id_list:
        # offer_id has 10 valid values, except -1 represent nan values of offer_id
        if offer_id not in valid_offer_id:
            continue
        unique_offer_id = groupby_offer_id.get_group(offer_id)
        df_units, units_count = cut_unique_offer_id_2_units(unique_offer_id)

        # informational offer_type without 'offer completed'
        if unique_offer_id.offer_type.unique()[0] == 'informational':
            get_dateset_from_informational_offer(df_units, units_count, transactions)

        else:
        #unique_offer_id.offer_type.unique()[0] in ['bogo', 'discount']:
            get_dateset_from_other_offer(df_units, units_count, transactions)
        #print("data for unique offer_id wrangled, time is {}" .format(time()-start))
    print("data for unique person wrangled, time is {}" .format(time()-start))

In [ ]:
def fill_offer_id_4_transaction(original_id, updated_id):
    '''Filling the offer_id for transactions, if they are related with an offer.
    Final result is '-1' or a list: [offer_id_1, offer_id_2,...]

    INPUT:
        original_id - (str) original is '-1'; after updated maybe a str with more values but separated with ',' (because there will be more than one transaction related with an offer)
        updated_id - (int) represent the related offer_id

    OUTPUT:
        new_value - (str) updated offer_id infomation of transactions
    '''
    if original_id == '-1':  #obejct(astype('str')): str '-1'
        new_value = updated_id
    else:
        # there is already at least one valid transaction related with an offer
        new_value = original_id+','+updated_id
    return new_value

In [ ]:
def get_dateset_from_informational_offer(df_units, units_count, transactions):
    # target_dataset and transcript_offer are global variables.
    '''
    DESCRIPTION:
    For the offer_id == 2 | 7 (informational_offer)

    Update transactions with related offer_id in the original dataset transcript_offer.

    Update target_dataset_list - (list), the structure of target_dataset_list is list of dicts with following keys:
        'person', 'offer_id', 'time_received', 'time_viewed', 'time_transaction','time_completed','amount_with_offer','label_effective_offer'

    INPUT:
        df_units - (list of DataFrame) transaction units from a unique_offer_id of unique_person
        units_count - (int) number of transaction units in df_units
        transactions - (DataFrame) 'event' is 'transaction' for a unique_person

    OUTPUT: None
    '''
    #!!!REMEMBER: it's already units, with 'offer received' or not
    #!!!REMEMBER: units_count can't be 0, since there is no nan in 'event'
    person   = df_units[0].person.unique()[0]  #.unique() returns a numpy.array
    offer_id = df_units[0].offer_id.unique()[0]
    # different offer_id has a different valid duration
    duration = df_units[0].duration.unique()[0]

    for i in range(units_count):
        df_unit = df_units[i]

        time_received = df_unit[df_unit.event=='offer received'].time.min()
        time_viewed = df_unit[df_unit.event=='offer viewed'].time.min()
        time_completed = df_unit[df_unit.event=='offer completed'].time.min()

        # init the transaction time
        # (after a valid transaction, the offer is finished, so there will be at most one transaction time)
        time_transaction = -1
        # init the amount related to an offer
        amount_with_offer = 0
        # init the label of effective_offer
        label_effective_offer = -1

        # FLAG of 'offer received'
        is_received = (df_unit[df_unit.event=='offer received'].shape[0]!=0)

        if is_received:
            # at least one transaction exist
            if transactions.shape[0] != 0:
                transaction_time = np.array(transactions.time)
                time_begin = time_received
                time_end = time_received + duration

                is_valid_duration = (transaction_time >= time_begin) & (transaction_time <= time_end)
                valid_transactions = transactions[is_valid_duration]

                # update the 1st transaction, get the label_effective_offer
                if valid_transactions.shape[0] != 0:
                    # the 1st transaction is the valid transaction related with an offer
                    valid_transactions.head(1).loc[:, 'offer_id'] = offer_id
                    time_transaction = valid_transactions.head(1).time.min()

                    # get the data in original dataset transcript_offer, to update the offer_id of transaction with the related offer_id
                    global transcript_offer
                    valid_transactions_2b_labeled = transcript_offer.loc[valid_transactions.index]

                    # update the offer_id of transaction in transcript_offer
                    valid_transactions_2b_labeled['offer_id'] = valid_transactions_2b_labeled['offer_id'].apply(fill_offer_id_4_transaction, args=(offer_id,))
                    transcript_offer.update(valid_transactions_2b_labeled)

                    label_effective_offer = 1
                    amount_with_offer = valid_transactions.head(1).amount.sum()

            # received but without transaction
            else:
                label_effective_offer = 0

        # update the target_dataset
        global target_dataset_list
        target_dict = {
                    "person":   person,
                    "offer_id": offer_id,
                    "time_received": time_received,
                    "time_viewed": time_viewed,
                    "time_transaction": time_transaction,
                    "time_completed": time_completed,
                    "amount_with_offer": amount_with_offer,
                    "label_effective_offer": label_effective_offer}
        target_dataset_list.append(target_dict)


In [ ]:
def get_dateset_from_other_offer(df_units, units_count, transactions):
    '''
    DESCRIPTION:
    For the offer_id != (2 & 7)
    (REMEMBER to exclude offer_id == -1)

    Update transactions with related offer_id in the original dataset transcript_offer.

    Update target_dataset_list - (list), the structure of target_dataset_list is list of dicts with following keys:
        'person', 'offer_id', 'time_received', 'time_viewed', 'time_transaction','time_completed','amount_with_offer','label_effective_offer'

    INPUT:
        df_units - (list of DataFrame) transaction units from a unique_offer_id of unique_person
        units_count - (int) number of transaction units in df_units
        transactions - (DataFrame) 'event' is 'transaction' for a unique_person

    OUTPUT: None
    '''
    #!!!REMEMBER: it's already units, with 'offer received' or not
    #!!!REMEMBER: units_count can't be 0, since there is no nan in 'event'
    person   = df_units[0].person.unique()[0]  #.unique() returns a numpy.array
    offer_id = df_units[0].offer_id.unique()[0]
    # different offer_id has a different valid duration
    duration = df_units[0].duration.unique()[0]

    for i in range(units_count):
        df_unit = df_units[i]

        time_received = df_unit[df_unit.event=='offer received'].time.min()
        time_viewed = df_unit[df_unit.event=='offer viewed'].time.min()
        time_completed = df_unit[df_unit.event=='offer completed'].time.min()

        # init the transaction time with a empty list
        # (there will be more than one valid transaction time)
        time_transaction = ""
        # init the amount related to an offer
        amount_with_offer = 0
        # init the label of effective_offer
        label_effective_offer = -1

        # FLAG of 'offer received'
        is_received = (df_unit[df_unit.event=='offer received'].shape[0]!=0)
        # FLAG of 'offer completed'
        is_completed = (df_unit[df_unit.event=='offer completed'].shape[0]!=0)

        if is_received:
            if is_completed:
                #REMEMBER: to be completed, there must be transaction(s)
                transaction_time = np.array(transactions.time)

                #valid transaction(s) exist between 'offer received' and 'offer completed'
                is_valid_duration = (transaction_time >= time_received) & (transaction_time <= time_completed)
                valid_transactions = transactions[is_valid_duration]

                valid_transactions.loc[:, 'offer_id'] = offer_id

                # get the index of valid transactions, to update offer_id of transactions in the original dataset transcript_offer
                # with global declaration to update transcript_offer
                global transcript_offer
                # the transactions valid in duration of offer_id
                valid_transactions_2b_labeled = transcript_offer.loc[valid_transactions.index]
                valid_transactions_2b_labeled['offer_id']=valid_transactions_2b_labeled['offer_id'].apply(fill_offer_id_4_transaction, args=(offer_id,))
                transcript_offer.update(valid_transactions_2b_labeled)

                # update the label of effective_offer
                label_effective_offer = 1
                # get the amount under an offer_id
                amount_with_offer = valid_transactions.amount.sum()
                # there may be more than one valid transaction
                for time in valid_transactions.time.values.tolist():
                    time_transaction = time_transaction+','+str(time)


            else:
                # without 'offer completed'
                transaction_time = np.array(transactions.time)
                time_begin = time_received
                time_end = time_received + duration
                # transaction(s) in valid duration should be regarded as 'tried transaction(s)'
                is_valid_duration = (transaction_time >= time_begin) & (transaction_time <= time_end)
                valid_transactions = transactions[is_valid_duration]

                # transaction(s) in valid duration should be updated with the related offer_id in the dataset transcript_offer

                valid_transactions.loc[:, 'offer_id'] = offer_id
                valid_transactions_2b_labeled = transcript_offer.loc[valid_transactions.index]
                valid_transactions_2b_labeled['offer_id']=valid_transactions_2b_labeled['offer_id'].apply(fill_offer_id_4_transaction, args=(offer_id,))
                transcript_offer.update(valid_transactions_2b_labeled)

                label_effective_offer = 0 # tried but not completed
                amount_with_offer = valid_transactions.amount.sum()
                for time in valid_transactions.time.values.tolist():
                    time_transaction = time_transaction+','+str(time)

        # with global declaration, update target_dataset_list
        global target_dataset_list
        target_dict = {
                    "person":   person,
                    "offer_id": offer_id,
                    "time_received": time_received,
                    "time_viewed": time_viewed,
                    "time_transaction": time_transaction,
                    "time_completed": time_completed,
                    "amount_with_offer": amount_with_offer,
                    "label_effective_offer": label_effective_offer}
        target_dataset_list.append(target_dict)


In [ ]:
def cut_unique_offer_id_2_units(unique_offer_id):
    '''
    DESCRIPTION:
        The raw data is transcript of unique_offer_id in one unique_person. Since there may be more than one offer for this unique_offer_id. That's why we cut the transcript to independent pieces, and call it 'units'.

    INPUT:
        unique_offer_id - (DataFrame) transcript of unique_offer_id in one unique_person

    OUTPUT:
        df_units - (list of DataFrame) transaction units from a unique_offer_id of unique_person
        units_count - (int) number of transaction units in df_units
    '''
    events = unique_offer_id['event']
    index = events.index.values
    index_min = events.index.min()
    index_max = events.index.max()
    index_received = events[events=='offer received'].index

    df_units = []
    if len(index_received) == 0:
        units_count = 1
        df_unit = unique_offer_id
        df_units.append(df_unit)

    elif index_received[0] == index_min:
        units_count = len(index_received)
        #当units_count=1时？？
        if units_count == 1:
            df_unit = unique_offer_id
            df_units.append(df_unit)

        else:
            for i in range(units_count - 1):
                df_unit = unique_offer_id[(index >= index_received[i]) & (index < index_received[i+1])]
                df_units.append(df_unit)
            df_unit = unique_offer_id[(index >= index_received[i+1]) & (index <= index_max)]
            df_units.append(df_unit)

    else:
        units_count = len(index_received)+1
        df_unit = unique_offer_id[(index >= index_min) & (index < index_received[0])]
        df_units.append(df_unit)
        #当units_count=2时？？
        if units_count == 2:
            df_unit = unique_offer_id[(index >= index_received[0]) & (index <= index_max)]
            df_units.append(df_unit)
        for i in range(1, units_count - 1):
            df_unit = unique_offer_id[(index >= index_received[i]) & (index < index_received[i+1])]
            df_units.append(df_unit)
        df_unit = unique_offer_id[(index >= index_received[i+1]) & (index <= index_max)]
        df_units.append(df_unit)

    return df_units, units_count

In [ ]:
# same process as using class
person_ids = transcript_offer.person.unique()
for person_id in person_ids:
    unique_person = transcript_offer[transcript_offer['person']==person_id]
    transactions = unique_person[unique_person.event=='transaction']

    groupby_offer_id = unique_person.groupby(['offer_id'])
    offer_id_list = unique_person.offer_id.unique()

    get_dateset_from_unique_offer_id(groupby_offer_id, offer_id_list, transactions)

In [ ]:
# change target_dataset_list(list of dicts) to DataFrame
target_dataset = pd.DataFrame(target_dataset_list, 
                              columns=['person', 'offer_id', 'time_received', 'time_viewed', 'time_transaction','time_completed','amount_with_offer','label_effective_offer'])
transcript_offer  # already wrangled

## <a class="anchor" id="Save-Data">3.2. [Save the wrangled data](#Table-Start)</a>

```python
target_dataset.to_csv('wrangled_target_dataset.csv')
transcript_offer.to_csv('wrangled_transcript_offer.csv')
```

# <a class="anchor" id="Wrangled-Data">IV. [Explore the wrangled Data](#Table-Start)(preliminary)</a>

In [ ]:
'''
### Alternative: load in the two wrangled dataset

transcript_offer = pd.read_csv('./wrangled_transcript_offer.csv', dtype={'person': int})
# recover to original dataset: index is the same
transcript_offer.index = transcript_offer.iloc[:, 0].values
del transcript_offer['Unnamed: 0']

target_dataset = pd.read_csv('./wrangled_target_dataset.csv', dtype={'person': int, 'offer_id': str})
# recover to original dataset: index is the same
target_dataset.index = target_dataset.iloc[:, 0].values
del target_dataset['Unnamed: 0']
'''

## 4.1 Basic information

In [43]:
target_dataset.head()

,person,offer_id,time_received,time_viewed,time_transaction,time_completed,amount_with_offer,label_effective_offer
0,3.0,3,0.0,0.25,",5.5",5.50,19.89,1
1,3.0,7,7.0,9.00,9.25,NaN,19.67,1
2,3.0,0,17.0,17.00,",21.25",21.25,21.72,1
3,3.0,8,21.0,24.25,",21.25",21.25,21.72,1
4,5.0,9,0.0,0.75,,NaN,0.00,0


In [44]:
target_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66501 entries, 0 to 66500
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   person                 66501 non-null  float64
 1   offer_id               66501 non-null  object 
 2   time_received          66501 non-null  float64
 3   time_viewed            49860 non-null  float64
 4   time_transaction       66501 non-null  object 
 5   time_completed         32000 non-null  float64
 6   amount_with_offer      66501 non-null  float64
 7   label_effective_offer  66501 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 4.1+ MB


In [45]:
transcript_offer.head()

,person,event,time,amount,offer_id,duration,offer_type
0,3.0,offer received,0.0,0.0,3,7.0,bogo
1,5.0,offer received,0.0,0.0,9,7.0,discount
2,8.0,offer received,0.0,0.0,8,5.0,bogo
3,12.0,offer received,0.0,0.0,2,4.0,informational
4,13.0,offer received,0.0,0.0,4,10.0,discount


In [46]:
transcript_offer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272762 entries, 0 to 272761
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   person      272762 non-null  float64
 1   event       272762 non-null  object 
 2   time        272762 non-null  float64
 3   amount      272762 non-null  float64
 4   offer_id    272762 non-null  object 
 5   duration    148805 non-null  float64
 6   offer_type  148805 non-null  object 
dtypes: float64(4), object(3)
memory usage: 26.6+ MB


## 4.2 Modiefy the wrangled dataset

In [47]:
unique_person_list_init = transcript_offer.person.unique()
unique_person_list_wrangled = target_dataset.person.unique()

person_id_disappear = np.setdiff1d(transcript_offer.person.unique(), target_dataset.person.unique() , assume_unique=True)
person_id_disappear

array([ 5639., 15391.,   872., 14763.,  6789.])

**FOUND**：
- There are 5 person_id reduced

In [48]:
transcript_offer[transcript_offer.person.isin(person_id_disappear)]
# offer_id = -1, means transaction without an offer

,person,event,time,amount,offer_id,duration,offer_type
14272,5639.0,transaction,0.25,4.74,-1,NaN,NaN
19395,15391.0,transaction,0.75,16.62,-1,NaN,NaN
61614,872.0,transaction,7.25,0.65,-1,NaN,NaN
77551,872.0,transaction,9.25,1.91,-1,NaN,NaN
95136,14763.0,transaction,12.75,17.51,-1,NaN,NaN
96434,5639.0,transaction,13.25,5.09,-1,NaN,NaN
97362,6789.0,transaction,13.50,22.75,-1,NaN,NaN
97802,15391.0,transaction,13.50,27.60,-1,NaN,NaN
98606,14763.0,transaction,13.75,28.70,-1,NaN,NaN
126518,6789.0,transaction,15.75,27.20,-1,NaN,NaN


In [49]:
# more details for 5 person
profile_cleaned[profile_cleaned.id.isin(person_id_disappear)]

,gender,age,id,became_member_on,income
872,F,72,872,20171014,35000.0
5639,F,66,5639,20170929,34000.0
6789,F,55,6789,20160816,88000.0
14763,F,54,14763,20170106,72000.0
15391,M,91,15391,20151005,70000.0


In [50]:
profile_cleaned.income.mean()

65404.9915682968

**FOUND**:
1. Common features for these 5 person
    - more than 50 years old, maybe don't like modern media, enjoy the old day's life.
    - the enroll date is early.
So age and enroo date maybe two important factors that imply the way to interact with offer.
2. These 5 person have the same structure of data, easy to modify

In [51]:
# modify the target_dataset
modified_target_dataset = target_dataset.copy()
for person_id in person_id_disappear:
    offer_id = '-1'
    # these 5 person have the same structure of data
    # set time_transaction = -1(init value)
    # set label_effective_offer = -2
    to_append = [person_id, offer_id, np.nan, np.nan, -1, np.nan, 0, -2]
    to_append_series = pd.Series(to_append, index = target_dataset.columns)
    
    # append to target_dataset
    modified_target_dataset = modified_target_dataset.append(to_append_series, ignore_index=True)

## 4.3 Save modified target_dataset
```python
modified_target_dataset.to_csv('modified_wrangled_target_dataset.csv')
```

# <a class="anchor" id="References">[References](#Table-Start)</a>
[1]https://intellipaat.com/community/32880/create-dummies-from-a-column-with-multiple-values-in-pandas

# Summary(for myself)
### 1. Key points (some errors easily happend)
1. Type of data
    - **offer_id**: changed int to obeject '0' '-1', so that can store more possible values, just for `event == 'transaction'`

2. []list {}dict are not hashable, with them `.unique()` can't be used. 
    - use str to replace list
    
    ```python
    for time in valid_transactions.time.values.tolist():
        time_transaction = time_transaction+','+str(time)               
    ```
3. global variables just for small programm

### 2. Usage of `class`
1. apply.(func, args=(,)) <br>
    func(**self**, ....)
    
### 3. Data IO
1. .to_csv(index=True) 第一列为index, 但是读取时提供RangeIndex,同时加一列Unnamed: 0
    - 如果index=False,原先的index就会消失不见 <br>
关键：如何转化Unnamed:0为index——恢复原先的表格数据
    ```python
    df.index = df.iloc[:, 0].values
    del df['Unnamed: 0']
    ```